# LogicalCircuit To Tket Circuit Conversion

In [ ]:
from LogicalQ.Logical import LogicalCircuit
from LogicalQ.Library.QECCs import steane_code

from pytket.extensions.quantinuum import QuantinuumBackend
from pytket.extensions.qiskit import qiskit_to_tk
from pytket.circuit.display import render_circuit_jupyter as draw

from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Decompose
from LogicalQ.Transpilation.UnBox import UnBoxTask
from LogicalQ.Transpilation.DecomposeIfElseOps import DecomposeIfElseOpsTask

In [ ]:
lqc = LogicalCircuit(1, **steane_code)

# Check encode
lqc.encode(0)

# Check transversal gates
lqc.x(0)
lqc.z(0)
lqc.y(0)

# Check complex gates
lqc.h(0)
lqc.s(0)
lqc.sdg(0)
lqc.t(0)
lqc.tdg(0)

# Check QEC cycle
lqc.append_qec_cycle()

# Check measurements
lqc.measure_all()

In [ ]:
# from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
# from qiskit.circuit.classical import expr

# def cbit_and(cbits, values):
#     result = expr.bit_not(cbits[0]) if values[0] == 0 else expr.lift(cbits[0])
#     for n in range(len(cbits)-1):
#         result = expr.bit_and(result, expr.bit_not(cbits[n+1])) if values[n+1] == 0 else expr.bit_and(result, cbits[n+1])
#     return result

# def cbit_xor(cbits):
#     result = expr.lift(cbits[0])
#     for n in range(len(cbits)-1):
#         result = expr.bit_xor(result, cbits[n+1])
#     return result

# qreg = QuantumRegister(1, "qreg")
# qreg_setter = QuantumRegister(2, "qsetter")
# creg_meas = ClassicalRegister(2, "creg_meas")
# creg_A = ClassicalRegister(2, "cintermediate_state")

# lqc = QuantumCircuit(qreg, qreg_setter, creg_meas, creg_A)
# lqc.x(qreg_setter[1])

# # with lqc.if_test(cbit_and([creg_meas[0], creg_meas[1]], [1,1])) as _else_left:
# with lqc.if_test(cbit_xor([creg_meas[x] for x in [0,1]])) as _else:
#     lqc.measure(qreg_setter[1], creg_A[0])
#     lqc.measure(qreg_setter[0], creg_A[1])
# with _else:
#     lqc.measure(qreg_setter[1], creg_A[0])
#     lqc.measure(qreg_setter[0], creg_A[1])

# lqc.measure(qreg, creg_meas[0])

In [ ]:
pm = PassManager([UnBoxTask(), DecomposeIfElseOpsTask(), Decompose()])
lqc = pm.run(lqc)

In [ ]:
lqc.draw("mpl")

In [ ]:
circuit = qiskit_to_tk(lqc)
print(circuit, type(circuit))

In [ ]:
# draw(circuit)

In [ ]:
# backend = QuantinuumBackend(device_name="H2-1")
# circuit_compiled = backend.get_compiled_circuit(circuit, optimisation_level=1)
# cost = backend.cost(circuit_compiled, n_shots=1)